## Blackjack Simulation: Part 1

Welcome to this Jupyter Notebook experiment on simulating a single player hand in the game of Blackjack.

In this notebook, we will focus on simulating (A) the distribution of outcomes on the two cards, and (B) standing on different values (targets ranging from 12 to 21).

Through Python, we aim to gain insights into the probabilities and strategies involved in the game of Blackjack.

This is just the beginning of our exploration (hence the name "Part 1"). In the future, we plan to expand our Monte Carlo simulation to include other factors, such as multiplayer scenarios. Let's dive in and uncover the secrets of Blackjack together!

In [1]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Implementation:

In [2]:
class Deck:
    def __init__(self, num_decks=1):
        # Cards as integers, representing the value of the card in the game of blackjack
        _spades = [11, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10 , 10, 10]
        _hearts = [11, 2, 3, 4, 5, 6, 7, 8, 9, 10 , 10, 10, 10]
        _clubs = [11, 2, 3, 4, 5, 6, 7, 8, 9, 10 , 10, 10, 10]
        _diamonds = [11, 2, 3, 4, 5, 6, 7, 8, 9, 10 , 10, 10, 10]

        self.deck = []
        for i in range(num_decks):
            self.deck = _spades + _hearts + _clubs + _diamonds + self.deck
        random.shuffle(self.deck)
    
    def deal(self):
        return self.deck.pop()
    
    def __len__(self):
        return len(self.deck)
    
    def __str__(self):
        return str(self.deck)

In [3]:
class Hand:
    def __init__(self, cards):
        self.cards = cards
        
    def get_value(self):
        # Calculate the value of the hand

        total = sum(self.cards)
        _aces = self.cards.count(11) # temp var, init=no. of aces in the hand
        
        while total > 21 and _aces > 0:
            total -= 10
            _aces -= 1 # Update temp var, decrement no. of aces in hand

        return total

    def hit(self, card):
        self.cards.append(card)

    def __str__(self):
        return str(self.cards)

## Simulation1
- Blackjack pays 3 to 2
- Dealer must stand on soft 17 and must draw to 16
- No splits

In [8]:
# Simulation: Dealing the first two cards, no hit, just stand
trials = 1000000

player_equity = 0
dealer_equity = 0

deck = Deck(6)
for i in range(trials):
    # Save states for assertion
    _player_equity = player_equity
    _dealer_equity = dealer_equity

    # Shuffle the deck if there are less than 52 cards left
    if len(deck) < 52:
        deck = Deck(6)

    # Initialize hands and deal the first two cards
    dealer_hand = Hand([])
    my_hand = Hand([]) 
    my_hand.hit(deck.deal())
    dealer_hand.hit(deck.deal())
    my_hand.hit(deck.deal())
    dealer_hand.hit(deck.deal())

    # Player: Stand on 17 or higher
    while my_hand.get_value() < 17:
        my_hand.hit(deck.deal())

    if my_hand.get_value() <= 21:
        
        # Dealer draws to 16 
        while dealer_hand.get_value() < 16:
            dealer_hand.hit(deck.deal())
        
        # Dealer stands on soft 17
        if dealer_hand.get_value() == 17 and 11 in dealer_hand.cards:
            pass
        elif dealer_hand.get_value() == 17:
            dealer_hand.hit(deck.deal())
        
    if my_hand.get_value() > 21:
        dealer_equity += 1 # Dealer wins
        player_equity -= 1
    elif my_hand.get_value() <= 21:    
        # Pushs = either (A) both have blackjack or (B) both have the same value
        _push1 = my_hand.get_value() == 21 and len(my_hand.cards) == 2 and dealer_hand.get_value() == 21 and len(dealer_hand.cards) == 2
        _push2 = my_hand.get_value() == dealer_hand.get_value()

        if _push1 or _push2:
            pass
        elif my_hand.get_value() == 21 and len(my_hand.cards) == 2:
            player_equity += 1.5 # Player wins, blackjack
            dealer_equity -= 1.5
        elif dealer_hand.get_value() > 21 or my_hand.get_value() > dealer_hand.get_value():
            player_equity += 1 # Player wins
            dealer_equity -= 1
        elif dealer_hand.get_value() > my_hand.get_value():
            dealer_equity += 1 # Dealer wins
            player_equity -= 1
    
    assert _player_equity != player_equity or _dealer_equity != dealer_equity or _push1 or _push2, "Something went wrong"

print("Player equity: ", player_equity )
print("Dealer equity: ", dealer_equity )
print("Player's E(r) = ", player_equity/trials) # each trial requires wagering 1 unit of bet

Player equity:  -29538.5
Dealer equity:  29538.5
Player's E(r) =  -0.0295385
